In [129]:
import pandas as pd
import numpy as np
import copy
import sys
from datetime import datetime
import argparse
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as auprc
from sklearn.metrics import roc_auc_score as auc_score
import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, Dense, GRU, Lambda, Permute, LSTM
from keras.models import Model
from interpolation_layer import single_channel_interp, cross_channel_interp
import warnings
from sklearn.preprocessing import KBinsDiscretizer
warnings.filterwarnings("ignore")
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import  MaxAbsScaler, MinMaxScaler

In [130]:
admission = pd.read_csv('Dataset_bench/with_age.csv')
admission = admission[["HADM_ID", "ADMISSION_TYPE", "LOS"]]
admission = admission[admission['LOS'] > 1]
admission = admission[admission['LOS'] < 10]
# admission = admission[["HADM_ID", "ADMISSION_TYPE", "EXPIRE_FLAG"]]
# admission['LOS'] = admission['LOS']
admission.head()

,HADM_ID,ADMISSION_TYPE,LOS
0,165315,EMERGENCY,1.144444
1,152223,ELECTIVE,5.496528
2,124321,EMERGENCY,6.768056
3,161859,EMERGENCY,2.856944
4,129635,EMERGENCY,3.534028


In [131]:
# heart rate
item_hr = pd.read_csv('gru_features/HR.csv')
item_hr['CHARTTIME'] = pd.to_datetime(item_hr['CHARTTIME']).dt.tz_localize(None)
item_hr = item_hr[item_hr['VALUE'].notna()]

In [132]:
# dbp
item_dbp = pd.read_csv('gru_features/dbp.csv')
item_dbp['CHARTTIME'] = pd.to_datetime(item_dbp['CHARTTIME']).dt.tz_localize(None)
item_dbp = item_dbp[item_dbp['VALUE'].notna()]

In [133]:
# glucose
item_glucose = pd.read_csv('gru_features/HR.csv')
item_glucose['CHARTTIME'] = pd.to_datetime(item_glucose['CHARTTIME']).dt.tz_localize(None)
item_glucose = item_glucose[item_glucose['VALUE'].notna()]

In [134]:
# ph
item_ph = pd.read_csv('gru_features/ph.csv')
item_ph['CHARTTIME'] = pd.to_datetime(item_ph['CHARTTIME']).dt.tz_localize(None)
item_ph = item_ph[item_ph['VALUE'].notna()]

In [135]:
# rr
item_rr = pd.read_csv('gru_features/RR.csv')
item_rr['CHARTTIME'] = pd.to_datetime(item_rr['CHARTTIME']).dt.tz_localize(None)
item_rr = item_rr[item_rr['VALUE'].notna()]

In [136]:
# sbp
item_sbp = pd.read_csv('gru_features/sbp.csv')
item_sbp['CHARTTIME'] = pd.to_datetime(item_sbp['CHARTTIME']).dt.tz_localize(None)
item_sbp = item_sbp[item_sbp['VALUE'].notna()]

In [137]:
# spo2
item_spo2 = pd.read_csv('gru_features/SpO2.csv')
item_spo2['CHARTTIME'] = pd.to_datetime(item_spo2['CHARTTIME']).dt.tz_localize(None)
item_spo2 = item_spo2[item_spo2['VALUE'].notna()]

In [138]:
# temp
item_temp = pd.read_csv('gru_features/temp.csv')
item_temp['CHARTTIME'] = pd.to_datetime(item_temp['CHARTTIME']).dt.tz_localize(None)
item_temp = item_temp[item_temp['VALUE'].notna()]

In [139]:
# tgcs
item_tgcs = pd.read_csv('gru_features/tgcs.csv')
item_tgcs['CHARTTIME'] = pd.to_datetime(item_tgcs['CHARTTIME']).dt.tz_localize(None)
item_tgcs = item_tgcs[item_tgcs['VALUE'].notna()]

In [140]:
# uo
item_uo = pd.read_csv('gru_features/uo.csv')
item_uo['CHARTTIME'] = pd.to_datetime(item_uo['CHARTTIME']).dt.tz_localize(None)
item_uo = item_uo[item_uo['VALUE'].notna()]

In [141]:
scaler =  MinMaxScaler()
item_dbp[['VALUE']] = scaler.fit_transform(item_dbp[['VALUE']])
item_glucose[['VALUE']] = scaler.fit_transform(item_glucose[['VALUE']])
item_hr[['VALUE']] = scaler.fit_transform(item_hr[['VALUE']])
item_ph[['VALUE']] = scaler.fit_transform(item_ph[['VALUE']])
item_rr[['VALUE']] = scaler.fit_transform(item_rr[['VALUE']])
item_sbp[['VALUE']] = scaler.fit_transform(item_sbp[['VALUE']])
item_spo2[['VALUE']] = scaler.fit_transform(item_spo2[['VALUE']])
item_temp[['VALUE']] = scaler.fit_transform(item_temp[['VALUE']])
item_tgcs[['VALUE']] = scaler.fit_transform(item_tgcs[['VALUE']])
item_uo[['VALUE']] = scaler.fit_transform(item_uo[['VALUE']])

In [143]:
# label_lst = [1.00138889,  19.33532014, 118.87907366, 206.42569444]
cat1 = admission[admission['LOS'] < 3]
cat2 = admission[admission['LOS'].between(3, 7)]
cat3 = admission[admission['LOS'] > 7]
# lens = [cat1.shape[0]/admission.shape[0],
#         cat2.shape[0]/admission.shape[0],
#         cat3.shape[0]/admission.shape[0]]
# id1 = int(300 * 0.79)
# id2 = int(300 * 0.18)
# id3 = 300 - id1 - id2

# # X for label 3:
cat1_ids = cat1['HADM_ID'].tolist()
cat2_ids = cat2['HADM_ID'].tolist()
cat3_ids = cat3['HADM_ID'].tolist()

# random.shuffle(cat1_ids)
# random.shuffle(cat2_ids)
# random.shuffle(cat3_ids)

# needed_ids_cat1= cat1_ids[:100] + cat2_ids[:100] + cat3_ids[:100]
# needed_ids_cat2= cat1_ids[:100] + cat2_ids[:100] + cat3_ids[:100]
# needed_ids_cat3= cat1_ids[:100] + cat2_ids[:100] + cat3_ids[:100]

test_ids = cat1_ids[-5 :]+ cat2_ids[-5:]+ cat3_ids[-5:]

In [145]:
vitals_dict = {}
np.random.seed(10)
num_features = 10
max_length = 2000 # length of timestamp
gpu_num = 0
ref_points = 192
hid = 100
batch = 256
epoch = 50
hours_looks_ahead = 24
num_fold = 3
stds = []

In [146]:
def select_ids(df, ids):
    return df[df['HADM_ID'].isin(ids)]

In [121]:
# temp_dbp = select_ids(item_dbp)
# temp_glucose = select_ids(item_glucose)
# temp_hr = select_ids(item_hr)
# temp_ph = select_ids(item_ph)
# temp_rr = select_ids(item_rr)
# temp_sbp = select_ids(item_sbp)
# temp_spo2 = select_ids(item_spo2)
# temp_temp = select_ids(item_temp)
# temp_tgcs = select_ids(item_tgcs)
# temp_uo = select_ids(item_uo)

In [147]:
# return two lists of hadm_id: list of timestamps (ascending order), list of corresponding vals of the item
def time_val_toLst(df, hadmin_id):
    temp = df.loc[df['HADM_ID'] == hadmin_id].sort_values(by=['CHARTTIME'])
    if not temp.empty: 
        times = temp.groupby('HADM_ID')['CHARTTIME'].apply(list).tolist()[0]
        vals = temp.groupby('HADM_ID')['VALUE'].apply(list).tolist()[0]
        return [times, vals]
    else:
        return [] # if id does not have this item, return empty list

In [148]:
def flatten(df, los=hours_looks_ahead):
    a = np.full((len(df), num_features, max_length), -100) # initlizae all as missing
    timestamps = []
    for i in range(len(df)):
        l = []
    
        # find all the unique observed timestamps
        for j in range(num_features):
            ts_ij = df[i][j]
            if ts_ij != []:
                for ts in df[i][j][0]:
                    if ts not in l:
                        l.append(ts)
        l.sort()
        T = copy.deepcopy(l)
        TS = []
        for t in T:
            if (t - T[0]).total_seconds() / 3600 <= los:
                TS.append(t)
        timestamps.append(TS)
        for j in range(num_features):
            s_ij = df[i][j]
            if s_ij != []:
                ts_ij = s_ij[0] 
                c_max = len(ts_ij)
                c = 0
                for k in range(len(TS)):
                    if c < c_max:
                        ts_ijc = ts_ij[c] # cur ts
                        diff = abs(TS[k] - ts_ijc).seconds / 60 # difference between ts's
                        if TS[k] == ts_ijc or diff < 5:
                            try:
                                a[i,j,k] = s_ij[1][c] 
                            except:
                                a[i,j,k] = -100
                            c += 1
    return a, timestamps

In [149]:
def input_format(x, T):
    real_len = 200
    
    for i in range(len(T)):
        if len(T[i]) > real_len:
            T[i] = T[i][:real_len]

    x = a[:, :, :real_len]
    M = np.zeros_like(x)
    delta = np.zeros_like(x)

    for t in T:
        for i in range(1, len(t)):
            t[i] = (t[i] - t[0]).total_seconds() / 3600 # hours difference
        if len(t) != 0:
            t[0] = 0
    
    # count outliers and negative values as missing values
    # M = 0 indicates missing value
    # M = 1 indicates observed value
    # now since we have mask variable, we don't need -100
    M[x > 500] = 0
    x[x > 500] = 0.0
    M[x < 0] = 0
    x[x < -30] = 0.0
    M[x > 0] = 1

    for i in range(num_features):
        for j in range(x.shape[0]):
            for k in range(len(T[j])):
                delta[j, i, k] = T[j][k]
    return x, M, delta

In [150]:
def missing_mean(M, x):
    counts = np.sum(np.sum(M, axis=2), axis=0)
    counts = [c + np.finfo(float).tiny for c in counts]
    mean_values = np.sum(np.sum(x*M, axis=2), axis=0)/counts
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if np.sum(M[i, j]) == 0:
                M[i, j, 0] = 1
                x[i, j, 0] = mean_values[j]
    return 

In [151]:
def hold_out(mask, perc=0.2):
    """To implement the autoencoder component of the loss, we introduce a set
    of masking variables mr (and mr1) for each data point. If drop_mask = 0,
    then we removecthe data point as an input to the interpolation network,
    and includecthe predicted value at this time point when assessing
    the autoencoder loss. In practice, we randomly select 20% of the
    observed data points to hold out from
    every input time series."""
    drop_mask = np.ones_like(mask)
    drop_mask *= mask
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            count = np.sum(mask[i, j], dtype='int')
            if int(0.20*count) > 1:
                index = 0
                r = np.ones((count, 1))
                b = np.random.choice(count, int(0.20*count), replace=False)
                r[b] = 0
                for k in range(mask.shape[2]):
                    if mask[i, j, k] > 0:
                        drop_mask[i, j, k] = r[index]
                        index += 1
    return drop_mask

In [152]:
# # create the vitals dictionary: {id: [item1, item2, item3, item4, item5]}
# # item1 = [list of timestamps, list of vals]
# # only use the first 200 hadm_ids as sample

# # 220045, spo2, rr,  
# # needed_ids = adm_ids[:num_ids]
# for adm_id in needed_ids_cat1:
#     vitals_dict[adm_id] = [time_val_toLst(temp_sbp, adm_id)]
#     vitals_dict[adm_id].append(time_val_toLst(temp_glucose, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_hr, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_ph, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_rr, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_sbp, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_spo2, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_temp, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_tgcs, adm_id))
#     vitals_dict[adm_id].append(time_val_toLst(temp_uo, adm_id))

In [153]:
stds = []
if feature_set == 2:
    stds.append(temp_dbp['VALUE'].std())
    stds.append(temp_glucose['VALUE'].std())
    stds.append(temp_hr['VALUE'].std())
    stds.append(temp_ph['VALUE'].std())
    stds.append(temp_rr['VALUE'].std())
    stds.append(temp_sbp['VALUE'].std())
    stds.append(temp_spo2['VALUE'].std())
    stds.append(temp_temp['VALUE'].std())
    stds.append(temp_tgcs['VALUE'].std())
    stds.append(temp_uo['VALUE'].std())

In [154]:
def y_trans(val):
    if val < 3:
        return 1.0
    else:
        return 0.0

In [155]:
# vitals = [vitals_dict[x] for x in needed_ids_cat1] # hadm_id(los>=48h): all the vitals values
# label = [admission[admission['HADM_ID'] == adm_id]['EXPIRE_FLAG'].values[0] for adm_id in needed_ids_cat1]

In [156]:
# # vitals = [vitals_dict[x] for x in needed_ids] # hadm_id(los>=48h): all the vitals values
# kb = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='kmeans')
# label = kb.fit_transform(admission.LOS.to_numpy().reshape(-1,1))
# # label = admission[admission['HADM_ID'].isin(needed_ids)].LOS.tolist()
# # label = [y_trans(l) for l in label]
# print(kb.bin_edges_)

In [157]:
# a, ts = flatten(vitals, hours_looks_ahead)
# x, m, T = input_format(a, ts)
# missing_mean(m, x)
# X = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
# y = np.array(label)
# print(X.shape, y.shape)
# timestamp = X.shape[2]
# num_features = X.shape[1] // 4

In [158]:
# [17.3, 22, 22.45, 2.32, 23.41，5.7, 3.33, 14.1]
def customloss(ytrue, ypred):
    """ Autoencoder loss
    """
    num_fs = num_features
    # standard deviation of each feature mentioned in paper for MIMIC_III data
    wc = np.array(stds)
    print(wc.shape, ytrue.shape)
    wc.shape = (1, num_fs)
    y = ytrue[:, :num_fs, :]
    m2 = ytrue[:, 3*num_fs:4*num_fs, :]
    m2 = 1 - m2
    m1 = ytrue[:, num_fs:2*num_fs, :]
    m = m1*m2
    ypred = ypred[:, :num_fs, :]
    x = (y - ypred)*(y - ypred)
    x = x*m
    count = tf.reduce_sum(m, axis=2)
    count = tf.where(count > 0, count, tf.ones_like(count))
    x = tf.reduce_sum(x, axis=2)/count
    x = x/(wc**2)  # dividing by standard deviation
    x = tf.reduce_sum(x, axis=1)/num_fs
    return tf.reduce_mean(x)

In [159]:
def interp_net():
    num_fs = num_features
    if gpu_num > 1:
        dev = "/cpu:0"
    else:
        dev = "/gpu:0"
    with tf.device(dev):
        main_input = Input(shape=(4*num_fs, timestamp), name='input')
        sci = single_channel_interp(ref_points, hours_looks_ahead)
        cci = cross_channel_interp()
        interp = cci(sci(main_input))
        reconst = cci(sci(main_input, reconstruction=True),
                      reconstruction=True)
        aux_output = Lambda(lambda x: x, name='aux_output')(reconst)
        z = Permute((2, 1))(interp)
        z = GRU(hid, activation='tanh', recurrent_dropout=0.2, dropout=0.2)(z)
        main_output = Dense(1, activation='sigmoid', name='main_output')(z)
        orig_model = Model([main_input], [main_output, aux_output])
    if gpu_num > 1:
        model = multi_gpu_model(orig_model, gpus=gpu_num)
    else:
        model = orig_model
    print(orig_model.summary())
    return model

In [160]:
seed = 0
results = {}
results['loss'] = []
results['auc'] = []
results['acc'] = []
results['auprc'] = []
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0000, patience=20, verbose=0)
callbacks_list = [earlystop]

In [165]:
fold_size = 50
for num in range(5):
    start = num * fold_size
    end = (num+1) * fold_size
    needed_ids= cat1_ids[start:end] + cat2_ids[start:end] + cat3_ids[start:end]
    
    temp_dbp = select_ids(item_dbp, needed_ids)
    temp_glucose = select_ids(item_glucose, needed_ids)
    temp_hr = select_ids(item_hr, needed_ids)
    temp_ph = select_ids(item_ph, needed_ids)
    temp_rr = select_ids(item_rr, needed_ids)
    temp_sbp = select_ids(item_sbp, needed_ids)
    temp_spo2 = select_ids(item_spo2, needed_ids)
    temp_temp = select_ids(item_temp, needed_ids)
    temp_tgcs = select_ids(item_tgcs, needed_ids)
    temp_uo = select_ids(item_uo, needed_ids)
    
    vitals_dict = {}
    for adm_id in needed_ids:
        vitals_dict[adm_id] = [time_val_toLst(temp_sbp, adm_id)]
        vitals_dict[adm_id].append(time_val_toLst(temp_glucose, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_hr, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_ph, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_rr, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_sbp, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_spo2, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_temp, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_tgcs, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(temp_uo, adm_id))
    
    vitals = [vitals_dict[x] for x in needed_ids] # hadm_id(los>=48h): all the vitals values
    label = [admission[admission['HADM_ID'] == adm_id]['LOS'].values[0] for adm_id in needed_ids]
    label = [y_trans(l) for l in label]
    
    a, ts = flatten(vitals, hours_looks_ahead)
    x, m, T = input_format(a, ts)
    missing_mean(m, x)
    X = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
    y = np.array(label)
    timestamp = X.shape[2]
    num_features = X.shape[1] // 4
    print(X.shape, y.shape)
    
    i = 0
    kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
    for train, test in kfold.split(np.zeros(len(y)), y):
        print("Running Fold:", i+1)
        model1 = interp_net()  # re-initializing every time
        model1.compile(
            optimizer='adam',
            loss={'main_output': 'binary_crossentropy', 'aux_output': customloss},
            loss_weights={'main_output': 1., 'aux_output': 1.},
            metrics={'main_output': 'accuracy'})
        history1 = model1.fit(
            {'input': X[train]}, {'main_output': y[train], 'aux_output': X[train]},
            batch_size=75,
            callbacks=callbacks_list,
            nb_epoch=20,
            validation_split=0.20,
            verbose=2)
        y_pred = model1.predict(X[test], batch_size=batch)
        y_pred = y_pred[0]
        total_loss, score, reconst_loss, acc = model1.evaluate(
            {'input': X[test]},
            {'main_output': y[test], 'aux_output': X[test]},
            batch_size=batch,
            verbose=0)
        results['loss'].append(score)
        results['acc'].append(acc)
        results['auc'].append(auc_score(y[test], y_pred))
        results['auprc'].append(auprc(y[test], y_pred))
        print(results)
        i += 1

(150, 40, 200) (150,)
Running Fold: 1
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 40, 200)      0                                            
__________________________________________________________________________________________________
single_channel_interp_16 (singl multiple             10          input[0][0]                      
                                                                 input[0][0]                      
__________________________________________________________________________________________________
cross_channel_interp_16 (cross_ multiple             100         single_channel_interp_16[0][0]   
                                                                 single_channel_interp_16[1][0]   
_____________________________________________________

Train on 60 samples, validate on 15 samples
Epoch 1/20
 - 7s - loss: 2868141042434048.0000 - main_output_loss: 2868141042434048.0000 - aux_output_loss: 5.0905e-14 - main_output_accuracy: 0.4833 - val_loss: 0.6636 - val_main_output_loss: 0.6636 - val_aux_output_loss: 8.4954e-14 - val_main_output_accuracy: 0.9333
Epoch 2/20
 - 4s - loss: 21445407145984.0000 - main_output_loss: 21445407145984.0000 - aux_output_loss: 1.6291e-13 - main_output_accuracy: 0.4333 - val_loss: 0.6626 - val_main_output_loss: 0.6626 - val_aux_output_loss: 1.7800e-13 - val_main_output_accuracy: 0.9333
Epoch 3/20
 - 3s - loss: 2504932982784.0000 - main_output_loss: 2504932982784.0000 - aux_output_loss: 1.5590e-13 - main_output_accuracy: 0.4500 - val_loss: 0.6616 - val_main_output_loss: 0.6616 - val_aux_output_loss: 4.1668e-13 - val_main_output_accuracy: 0.9333
Epoch 4/20
 - 3s - loss: 142683439104.0000 - main_output_loss: 142683439104.0000 - aux_output_loss: 3.2500e-13 - main_output_accuracy: 0.4833 - val_loss: 0.660

Epoch 4/20
 - 3s - loss: 0.6777 - main_output_loss: 0.6777 - aux_output_loss: 2.3035e-12 - main_output_accuracy: 0.5333 - val_loss: 0.5931 - val_main_output_loss: 0.5931 - val_aux_output_loss: 1.5939e-13 - val_main_output_accuracy: 1.0000
Epoch 5/20
 - 3s - loss: 0.6731 - main_output_loss: 0.6731 - aux_output_loss: 2.3283e-12 - main_output_accuracy: 0.5833 - val_loss: 0.5893 - val_main_output_loss: 0.5893 - val_aux_output_loss: 1.1893e-13 - val_main_output_accuracy: 1.0000
Epoch 6/20
 - 3s - loss: 0.6619 - main_output_loss: 0.6619 - aux_output_loss: 2.2656e-12 - main_output_accuracy: 0.5667 - val_loss: 0.5861 - val_main_output_loss: 0.5861 - val_aux_output_loss: 1.4644e-13 - val_main_output_accuracy: 1.0000
Epoch 7/20
 - 2s - loss: 0.6440 - main_output_loss: 0.6440 - aux_output_loss: 2.3032e-12 - main_output_accuracy: 0.5833 - val_loss: 0.5833 - val_main_output_loss: 0.5833 - val_aux_output_loss: 7.4435e-14 - val_main_output_accuracy: 1.0000
Epoch 8/20
 - 2s - loss: 0.6525 - main_outpu

Epoch 8/20
 - 3s - loss: 7724155797504.0000 - main_output_loss: 7724155797504.0000 - aux_output_loss: 1.8400e-13 - main_output_accuracy: 0.5167 - val_loss: 0.5931 - val_main_output_loss: 0.5931 - val_aux_output_loss: 4.6819e-12 - val_main_output_accuracy: 1.0000
Epoch 9/20
 - 3s - loss: 8848259678208.0000 - main_output_loss: 8848259678208.0000 - aux_output_loss: 1.5629e-13 - main_output_accuracy: 0.5500 - val_loss: 0.5921 - val_main_output_loss: 0.5921 - val_aux_output_loss: 4.7008e-12 - val_main_output_accuracy: 1.0000
Epoch 10/20
 - 3s - loss: 363195269120.0000 - main_output_loss: 363195269120.0000 - aux_output_loss: 1.3515e-13 - main_output_accuracy: 0.5333 - val_loss: 0.5912 - val_main_output_loss: 0.5912 - val_aux_output_loss: 4.6873e-12 - val_main_output_accuracy: 1.0000
Epoch 11/20
 - 2s - loss: 8431502336.0000 - main_output_loss: 8431502336.0000 - aux_output_loss: 2.4094e-13 - main_output_accuracy: 0.5500 - val_loss: 0.5905 - val_main_output_loss: 0.5905 - val_aux_output_loss: 

In [72]:
history_dict = history1.history
history_dict.keys()

acc = history_dict['main_output_accuracy']
val_acc = history_dict['val_main_output_accuracy']
loss = history_dict['main_output_loss']
val_loss = history_dict['val_main_output_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, acc, 'bo', label='Training acc')
# b is for "solid blue line"
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

dict_keys(['val_loss', 'val_main_output_loss', 'val_aux_output_loss', 'val_main_output_accuracy', 'loss', 'main_output_loss', 'aux_output_loss', 'main_output_accuracy'])

In [74]:
results

{'loss': [0.6918002963066101,
  0.6785474419593811,
  0.7042328715324402,
  0.693600594997406,
  0.6852771639823914],
 'auc': [0.5258333333333334,
  0.6522916666666667,
  0.4489583333333333,
  0.32687499999999997,
  0.48458333333333337],
 'acc': [0.5899999737739563,
  0.6000000238418579,
  0.41999998688697815,
  0.6000000238418579,
  0.5699999928474426],
 'auprc': [0.6221316391249946,
  0.6957140841781153,
  0.5959189425365962,
  0.5390679788048209,
  0.6029748897967951]}

In [320]:
!mkdir -p saved_model
model.save('saved_model/my_model') 
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

model1 model2
ls: saved_model/my_model: No such file or directory
